In [1]:
pip install undetected-chromedriver lxml


In [9]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver
browser = uc.Chrome()

# URL inicial: Página de listados de Madrid
base_url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(base_url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception:
    print("No se encontró el botón de cookies o ya fue aceptado.")

# Lista para almacenar los datos
all_data = []

# Función para extraer datos de un inmueble dado su enlace
def extraer_datos_inmueble(enlace):
    try:
        browser.get(enlace)
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Precio
            precio_element = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'})
            precio = precio_element.text.split(' ')[0] if precio_element else "N/A"

            # Superficie construida
            superficie_element = soup_inmueble.find('span', {'class': 'features__value'})
            superficie_construida = superficie_element.text.split(' ')[0] if superficie_element else "N/A"

            # Última actualización
            ultima_actualizacion_element = soup_inmueble.find('p', {'class': 'last-update__date'})
            ultima_actualizacion = ultima_actualizacion_element.text if ultima_actualizacion_element else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'})
                    value = feature.find('span', {'class': 'features__value'})
                    if label and value:
                        features_list.append((label.get_text(strip=True), value.get_text(strip=True)))
            else:
                features_list = "N/A"

            # Certificado energético
            energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
            if energy_certificate:
                consumo_element = energy_certificate.find('div', {'class': 'energy-certificate__data'})
                consumo = consumo_element.find_all('span')[1].get_text(strip=True) if consumo_element else "N/A"
                
                emisiones_element = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})
                emisiones = emisiones_element[1].find_all('span')[1].get_text(strip=True) if len(emisiones_element) > 1 else "N/A"
            else:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': enlace,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Alquiler'
            }

            return data

        except Exception as e:
            print(f"Error al extraer datos de {enlace}: {e}")
            return None

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        return None

# Bucle para iterar sobre múltiples páginas de resultados
page_number = 1
max_pages = 200  # Ajustar este límite según las necesidades
while page_number <= max_pages:
    try:
        # Cargar la página actual
        page_url = f"{base_url}pagina-{page_number}/"
        browser.get(page_url)
        WebDriverWait(browser, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
        )

        # Obtener todos los enlaces de la página actual
        html = browser.page_source
        soup = bs(html, 'lxml')
        inmuebles = soup.find_all('div', {'class': 'ad-preview'})

        # Extraer datos de cada inmueble de la página
        for inmueble in inmuebles:
            enlace = inmueble.get('data-lnk-href')
            if enlace:
                enlace_completo = f"https://www.pisos.com{enlace}"
                data = extraer_datos_inmueble(enlace_completo)
                if data:
                    all_data.append(data)

                # Pausa para evitar ser bloqueado
                time.sleep(random.uniform(3, 5))

        # Imprimir el progreso
        print(f"Procesada la página {page_number}. Total de inmuebles recogidos: {len(all_data)}")

        # Pasar a la siguiente página
        page_number += 1

    except Exception as e:
        print(f"Error al procesar la página {page_number}: {e}")
        break

# Convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('pisos_data_todos_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


Procesada la página 1. Total de inmuebles recogidos: 31
Procesada la página 2. Total de inmuebles recogidos: 62
Procesada la página 3. Total de inmuebles recogidos: 93
Procesada la página 4. Total de inmuebles recogidos: 124
Procesada la página 5. Total de inmuebles recogidos: 155
Procesada la página 6. Total de inmuebles recogidos: 186
Procesada la página 7. Total de inmuebles recogidos: 217
Procesada la página 8. Total de inmuebles recogidos: 248
Procesada la página 9. Total de inmuebles recogidos: 279
Procesada la página 10. Total de inmuebles recogidos: 310
Procesada la página 11. Total de inmuebles recogidos: 341
Procesada la página 12. Total de inmuebles recogidos: 372
Procesada la página 13. Total de inmuebles recogidos: 403
Procesada la página 14. Total de inmuebles recogidos: 434
Procesada la página 15. Total de inmuebles recogidos: 465
Procesada la página 16. Total de inmuebles recogidos: 496
Procesada la página 17. Total de inmuebles recogidos: 527
Procesada la página 18. To

In [ ]:
df